<a href="https://colab.research.google.com/github/huangliyoua/masker/blob/master/EGFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
安装方法一：

! pip3 install torch torchvision  # 省事

安装方法二：

! uname -a  # 查看系统  x86_64 GNU/Linux
! python --version  # 查看python版本 Python 3.6.7
! nvcc --version # 查看cuda版本 release 9.2, V9.2.148
在pytoch下载页面找到合适的版本，进行安装

! pip install https://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
! pip install torchvision

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

## visdom 准备工作

--2021-05-05 03:09:07--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.222.240.112, 34.226.165.133, 52.6.97.115, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.222.240.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  13.5MB/s    in 1.0s    

2021-05-05 03:09:08 (13.5 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [ ]:
##启动visdom

get_ipython().system_raw('python3 -m visdom.server -port 8097 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('./ngrok authtoken 1ojSS32bQn4j0jRzmRL0qLLRIfW_6nRHUb7yBsPw7PvG4ensh')
get_ipython().system_raw('./ngrok http 8097 &')


import visdom
viz = visdom.Visdom(env='test')
viz.text('aaaa', win='test text')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
 "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


In [ ]:
import  torch
import  os, glob
import  random, csv

from    torch.utils.data import Dataset, DataLoader

from    torchvision import transforms
from    PIL import Image


class Pokemon(Dataset):

    def __init__(self, root, resize, mode):
        super(Pokemon, self).__init__()

        self.root = '/content/drive/MyDrive/PNG'
        self.resize = resize

        self.name2label = {} # "sq...":0
        for name in sorted(os.listdir(os.path.join('/content/drive/MyDrive/PNG'))):
            if not os.path.isdir(os.path.join('/content/drive/MyDrive/PNG', name)):
                continue

            self.name2label[name] = len(self.name2label.keys())

        print(self.name2label)

        # image, label
        self.images, self.labels = self.load_csv('images.csv')

        if mode=='train': # 60%
            self.images = self.images[:int(0.7*len(self.images))]
            self.labels = self.labels[:int(0.7*len(self.labels))]


        elif mode == 'test': # 20% = 80%->100%
            self.images = self.images[int(0.7*len(self.images)):]
            self.labels = self.labels[int(0.7*len(self.labels)):]





    def load_csv(self, filename):

        if not os.path.exists(os.path.join(self.root, filename)):
            images = []
            for name in self.name2label.keys():
                # 'pokemon\\mewtwo\\00001.png
                images += glob.glob(os.path.join(self.root, name, '*.png'))
                images += glob.glob(os.path.join(self.root, name, '*.jpg'))
                images += glob.glob(os.path.join(self.root, name, '*.jpeg'))

            # 1167, 'pokemon\\bulbasaur\\00000000.png'
            print(len(images), images)

            random.shuffle(images)
            with open(os.path.join(self.root, filename), mode='w', newline='') as f:
                writer = csv.writer(f)
                for img in images: # 'pokemon\\bulbasaur\\00000000.png'
                    name = img.split(os.sep)[-2]
                    label = self.name2label[name]
                    # 'pokemon\\bulbasaur\\00000000.png', 0
                    writer.writerow([img, label])
                print('writen into csv file:', filename)

        # read from csv file
        images, labels = [], []
        with open(os.path.join(self.root, filename)) as f:
            reader = csv.reader(f)
            for row in reader:
                # 'pokemon\\bulbasaur\\00000000.png', 0
                img, label = row
                label = int(label)

                images.append(img)
                labels.append(label)
        print(len(labels))
        assert len(images) == len(labels)

        return images, labels



    def __len__(self):

        return len(self.images)


    def denormalize(self, x_hat):

        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]

        # x_hat = (x-mean)/std
        # x = x_hat*std = mean
        # x: [c, h, w]
        # mean: [3] => [3, 1, 1]
        mean = torch.tensor(mean).unsqueeze(1).unsqueeze(1)
        std = torch.tensor(std).unsqueeze(1).unsqueeze(1)
        # print(mean.shape, std.shape)
        x = x_hat * std + mean

        return x


    def __getitem__(self, idx):
        # idx~[0~len(images)]
        # self.images, self.labels
        # img: 'pokemon\\bulbasaur\\00000000.png'
        # label: 0
        img, label = self.images[idx], self.labels[idx]

        tf = transforms.Compose([
            lambda x:Image.open(x).convert('RGB'), # string path= > image data
            transforms.Resize((int(self.resize*1.25), int(self.resize*1.25))),
            transforms.RandomRotation(15),
            transforms.CenterCrop(self.resize),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

        img = tf(img)
        label = torch.tensor(label)


        return img, label





def main():

    #import  visdom
    import  time
    import  torchvision

    #viz = visdom.Visdom()

    # tf = transforms.Compose([
    #                 transforms.Resize((64,64)),
    #                 transforms.ToTensor(),
    # ])
    # db = torchvision.datasets.ImageFolder(root='pokemon', transform=tf)
    # loader = DataLoader(db, batch_size=32, shuffle=True)
    #
    # print(db.class_to_idx)
    #
    # for x,y in loader:
    #     viz.images(x, nrow=8, win='batch', opts=dict(title='batch'))
    #     viz.text(str(y.numpy()), win='label', opts=dict(title='batch-y'))
    #
    #     time.sleep(10)


    db = Pokemon('EGFR', 64, 'train')
    #print(db[0])

    x,y = next(iter(db))
    print('sample:', x.shape, y.shape, y)

    #viz.image(db.denormalize(x), win='sample_x', opts=dict(title='sample_x'))

    #loader = DataLoader(db, batch_size=32, shuffle=True, num_workers=8)

   # for x,y in loader:
      #  viz.images(db.denormalize(x), nrow=8, win='batch', opts=dict(title='batch'))
      #  viz.text(str(y.numpy()), win='label', opts=dict(title='batch-y'))
#
      #  time.sleep(10)

if __name__ == '__main__':
    main()

{'mutant': 0, 'wild': 1}
1534
sample: torch.Size([3, 64, 64]) torch.Size([]) tensor(0)


In [ ]:
from    matplotlib import pyplot as plt
import  torch
from    torch import nn

class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item()
        return x.view(-1, shape)


def plot_image(img, label, name):

    fig = plt.figure()
    for i in range(6):
        plt.subplot(2, 3, i + 1)
        plt.tight_layout()
        plt.imshow(img[i][0]*0.3081+0.1307, cmap='gray', interpolation='none')
        plt.title("{}: {}".format(name, label[i].item()))
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [ ]:
import  torch
from    torch import optim, nn
import  visdom
## python -m visdom.server
import  torchvision
from    torch.utils.data import DataLoader
import torchvision
#from    pokemon import Pokemon
# from    resnet import ResNet18
from    torchvision.models import resnet18
from    torchvision.models import resnet50
from    torch.nn import functional as F
#from    utils import Flatten
import numpy as np
from sklearn.metrics import roc_auc_score

batchsz = 64
lr = 1e-3
epochs = 1000

device = torch.device('cuda')
torch.manual_seed(1234)


train_db = Pokemon('pokemon', 224, mode='train')

test_db = Pokemon('pokemon', 224, mode='test')
train_loader = DataLoader(train_db, batch_size=batchsz, shuffle=True)

test_loader = DataLoader(test_db, batch_size=batchsz)


viz = visdom.Visdom()

def evalute(model, loader):
    model.eval()
    
    correct = 0
    total = len(loader.dataset)
    predlist = []
    scorelist = []
    targetlist = []
    for x,y in loader:
        x,y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
            correct += torch.eq(pred, y).sum().float().item()
            score = F.softmax(logits, dim=1)
            targetcpu = y.long().cpu().numpy()  # 由GPU->CPU
            predlist = np.append(predlist, pred.cpu().numpy())

            scorelist = np.append(scorelist, score.cpu().numpy()[:, 1])
            targetlist = np.append(targetlist, targetcpu)

    return targetlist, scorelist, predlist,correct / total

def main():

    # model = ResNet18(5).to(device)
    trained_model = resnet18(pretrained=True)
    model = nn.Sequential(*list(trained_model.children())[:-1], #[b, 512, 1, 1]
                          Flatten(), # [b, 512, 1, 1] => [b, 512]
                          nn.Linear(512, 2)
                          ).to(device)
    # x = torch.randn(2, 3, 224, 224)
    # print(model(x).shape)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5,
 verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
    criteon = nn.CrossEntropyLoss()


    best_acc, best_epoch = 0, 0
    global_step = 0
    viz.line([0], [-1], win='loss', opts=dict(title='loss'))
    viz.line([0], [-1], win='val_acc', opts=dict(title='val_acc'))

    TP = 0
    TN = 0
    FN = 0
    FP = 0

    r_list = []
    p_list = []
    acc_list = []
    AUC_list = []


    for epoch in range(epochs):
        running_loss=0
        for step, (x,y) in enumerate(train_loader):

            # x: [b, 3, 224, 224], y: [b]
            x, y = x.to(device), y.to(device)

            model.train()
            logits = model(x)
            loss = criteon(logits, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print('loss',loss)
            running_loss += loss.item()
            if (step+1) % batchsz == 0:    # 每个epoch中step loss的均值

                now_loss = running_loss / batchsz # mini-batches 的平均损失率

                scheduler.step(now_loss) ##更新学习率
           # viz.line([optimizer], [global_step], win='学习率', update='append')
            viz.line([loss.item()], [global_step], win='loss', update='append')
            global_step += 1

        if epoch % 1 == 0:
            targetlist, scorelist, predlist, val_acc = evalute(model, test_loader)
            #val_acc = evalute(model, test_loader)
            #print('pre',predlist,'tar',targetlist)
            TP = ((predlist == 1) & (targetlist == 1)).sum()
            TN = ((predlist == 0) & (targetlist == 0)).sum()
            FN = ((predlist == 0) & (targetlist == 1)).sum()
            FP = ((predlist == 1) & (targetlist == 0)).sum()

            print('TP=', TP, 'TN=', TN, 'FN=', FN, 'FP=', FP)
            print('TP+FP', TP + FP)
            p = TP / (TP + FP)
            print('precision', p)
            p = TP / (TP + FP)
            r = TP / (TP + FN)
            print('recall', r)
            F1 = 2 * r * p / (r + p)
            acc = (TP + TN) / (TP + TN + FP + FN)
            print('F1', F1)
            print('acc', acc)
            AUC = roc_auc_score(targetlist, scorelist)
            print('AUC', AUC)
            print('best acc:', best_acc, 'best epoch:', best_epoch)

        if val_acc> best_acc:
                best_epoch = epoch
                best_acc = val_acc

                torch.save(model.state_dict(), 'best.mdl')

                viz.line([val_acc], [global_step], win='val_acc', update='append')



    print('best acc:', best_acc, 'best epoch:', best_epoch)

    model.load_state_dict(torch.load('best.mdl'))
    print('loaded from ckpt!')

    test_acc = evalute(model, test_loader)
    print('test acc:', test_acc)





if __name__ == '__main__':
    main()


Setting up a new session...


流式输出内容被截断，只能显示最后 5000 行内容。
TP= 201 TN= 240 FN= 15 FP= 5
TP+FP 206
precision 0.9757281553398058
recall 0.9305555555555556
F1 0.9526066350710901
acc 0.9566160520607375
AUC 0.9921957671957672
best acc: 0.9696312364425163 best epoch: 169
loss tensor(0.0008, device='cuda:0', grad_fn=<NllLossBackward>)
loss tensor(5.2208e-06, device='cuda:0', grad_fn=<NllLossBackward>)
loss tensor(4.0158e-05, device='cuda:0', grad_fn=<NllLossBackward>)
loss tensor(5.7480e-06, device='cuda:0', grad_fn=<NllLossBackward>)
loss tensor(2.0755e-05, device='cuda:0', grad_fn=<NllLossBackward>)
loss tensor(2.3626e-05, device='cuda:0', grad_fn=<NllLossBackward>)
loss tensor(1.3740e-05, device='cuda:0', grad_fn=<NllLossBackward>)
loss tensor(2.7159e-05, device='cuda:0', grad_fn=<NllLossBackward>)
loss tensor(6.5414e-05, device='cuda:0', grad_fn=<NllLossBackward>)
loss tensor(2.6946e-05, device='cuda:0', grad_fn=<NllLossBackward>)
loss tensor(0.0004, device='cuda:0', grad_fn=<NllLossBackward>)
loss tensor(4.0639e-05, de

KeyboardInterrupt: ignored